In [1]:
import pandas as pd

In [87]:
FILTER_METAL = 'Silver'

## Read csv files

In [233]:
slcsp_df = pd.read_csv("slcsp.csv")
slcsp_df.head()

,zipcode,rate
0,64148,255.2
1,67118,NaN
2,40813,NaN
3,18229,NaN
4,51012,NaN


In [226]:
zips_df = pd.read_csv("zips.csv")

In [227]:
plans_df = pd.read_csv("plans.csv")
plans_df = plans_df[plans_df.metal_level == FILTER_METAL]
plans_df

,plan_id,state,metal_level,rate,rate_area
0,74449NR9870320,GA,Silver,298.62,7
1,26325VH2723968,FL,Silver,421.43,60
5,36956VI7724021,TX,Silver,310.12,11
12,72190DW3284384,AZ,Silver,353.37,4
17,22521KG6880321,GA,Silver,285.76,13
...,...,...,...,...,...
22233,32506NU6877376,SC,Silver,258.60,27
22234,27789PB4260586,WI,Silver,373.28,12
22235,82645LF4472021,WI,Silver,310.74,11
22236,07150US9925620,AZ,Silver,368.50,2


## Merge SLCSP and Zip dataframes to create new merged dataframe
### Remove duplicates in dataframe

In [249]:
merged_df = pd.merge(slcsp_df[['zipcode']], zips_df[['zipcode','state','rate_area']], on="zipcode")
merged_df = merged_df.drop_duplicates()
merged_df = merged_df.set_index('zipcode')
merged_df

,state,rate_area
zipcode,,
64148,MO,3
67118,KS,6
40813,KY,8
18229,PA,3
51012,IA,3
...,...,...
20047,DC,1
47452,IN,13
47452,IN,15


In [250]:
merged_df['count'] = merged_df.groupby(['zipcode']).count().rename(columns={"rate_area": "count"})[["count"]]
merged_df

,state,rate_area,count
zipcode,,,
64148,MO,3,1
67118,KS,6,1
40813,KY,8,1
18229,PA,3,1
51012,IA,3,1
...,...,...,...
20047,DC,1,1
47452,IN,13,3
47452,IN,15,3


In [251]:
zipgroup = merged_df.iloc[0]
zipgroup

state        MO
rate_area     3
count         1
Name: 64148, dtype: object

In [215]:
def get2ndLowestRateByZip(zip_group):
    # pull rates by state, rate_area and metal_level
    rates_df = plans_df[(plans_df.state == zip_group['state']) & (plans_df.rate_area == zip_group['rate_area']) & (plans_df.metal_level == FILTER_METAL)]
    
    # remove any duplicate rates and sort values
    rates_df = rates_df[['state', 'metal_level', 'rate_area', 'rate']].drop_duplicates().sort_values(by=['rate'])

    # extract 2nd lowest rate
    return rates_df.iloc[1,3]

In [235]:
zipgroup.name

64148

In [246]:
zipgroup = singles.iloc[0]
rate = get2ndLowestRateByZip(zipgroup)
slcsp_df.loc[slcsp_df['zipcode'] == zipgroup.name, "rate"]=rate

In [247]:
slcsp_df.head()

,zipcode,rate
0,64148,245.20
1,67118,212.35
2,40813,NaN
3,18229,231.48
4,51012,NaN
